# Cuaderno de Documentación: 06 - Orquestador Principal (`main.py`)

El script `scripts/main.py` es el punto de entrada principal para ejecutar el flujo completo del análisis de riesgos RAG. Su función es orquestar la ejecución secuencial de las diferentes etapas del proceso, llamando a las funciones apropiadas de los módulos auxiliares (`config.py`, `document_utils.py`, `vector_db_manager.py`, `rag_components.py`, y `report_utils.py`).

## Flujo de Ejecución Orquestado por `main.py`

La función `ejecutar_flujo_completo_analisis()` dentro de `main.py` sigue estos pasos:

1.  **Carga de Configuración e Inicialización de Directorios:**
    * Llama a `config.inicializar_directorios_datos()` para asegurar que todas las carpetas necesarias en `datos/` existan.
    * Llama a `config.configure_google_api()` para configurar la API de Gemini utilizando la `GEMINI_API_KEY` cargada desde el archivo `.env`.

2.  **Inicialización del Modelo de Embeddings:**
    * Llama a `vector_db_manager.get_embedding_function()` pasando el `EMBEDDING_MODEL_NAME` desde `config.py` para obtener la función que convertirá texto a vectores.

3.  **Gestión de la Base de Datos Vectorial:**
    * Llama a `vector_db_manager.crear_o_cargar_chroma_db()`. Esta función:
        * Utiliza la `embedding_function` obtenida en el paso anterior.
        * Utiliza las rutas y parámetros de `config.py` (`CHROMA_DB_PATH`, `DOCS_BASE_CONOCIMIENTO_PATH`, `CHUNK_SIZE`, `CHUNK_OVERLAP`).
        * Decide si crear una nueva base de datos o cargar una existente basándose en el flag `RECREAR_DB` de `config.py` y la existencia de la base de datos.

4.  **Configuración del LLM y la Cadena RAG:**
    * Llama a `rag_components.get_llm_instance()` para inicializar el modelo Gemini, pasando el `LLM_MODEL_NAME` y la `GEMINI_API_KEY` desde `config.py`.
    * Llama a `rag_components.crear_cadena_rag()`, proporcionando la instancia del LLM, la instancia de la base de datos vectorial (`vector_db`) y el número de documentos a recuperar (`K_RETRIEVED_DOCS` de `config.py`).

5.  **Procesamiento del Documento del Proyecto a Analizar:**
    * Construye la ruta completa al PDF del proyecto utilizando `PROYECTO_A_ANALIZAR_PATH` y `NOMBRE_PDF_PROYECTO_ANALIZAR` de `config.py`.
    * Llama a `document_utils.procesar_pdf_proyecto_para_analisis()` para cargar, fragmentar y concatenar el contenido del PDF del proyecto, aplicando también el truncamiento (`MAX_CHARS_PROYECTO`).

6.  **Ejecución de la Consulta RAG:**
    * Invoca la cadena `qa_chain` (obtenida en el paso 4) con la descripción del nuevo proyecto (obtenida en el paso 5) como la consulta (`query`).
    * Recupera el resultado del LLM (`resultado_analisis_llm`) y los documentos fuente (`fuentes_recuperadas_docs`).
    * Prepara los datos de las fuentes recuperadas para que sean serializables (guardando metadatos y un snippet del contenido).

7.  **Formateo y Guardado del Reporte Final:**
    * Llama a `report_utils.formatear_y_guardar_reporte()`, pasando la respuesta cruda del LLM, los datos de las fuentes recuperadas, y otros metadatos relevantes (nombre del proyecto, modelo LLM usado, ruta de salida de `config.py`). Esta función se encarga de parsear el JSON del LLM, asignar estados RAG y guardar el reporte final.

## Ejecución

Para ejecutar todo el proceso, el usuario simplemente necesita correr este script desde la línea de comandos (asegurándose de estar en el directorio `scripts/` y con el entorno virtual activado):
```bash
python main.py
```
El script imprimirá mensajes de progreso para cada etapa y, al finalizar, indicará dónde se ha guardado el reporte de análisis de riesgos.

A continuación, se muestra el contenido del script `main.py` para referencia.

In [ ]:
# Cargar el contenido del script main.py
%load ../scripts/main.py